In [210]:
%%timeit
#From Hw11
import pandas as pd
import numpy as np
aapl = pd.read_csv('c:/data/apple.2011.csv', sep=',')
aapl.columns = ['Date', 'Last','Change']
aapl.iloc[0,2] = 0
mean = pd.to_numeric(aapl['Change']).mean()
sd = pd.to_numeric(aapl['Change']).std()
aapl20 = aapl['Last'].iloc[-1]
def get20day(last):
    for i in range(1,20):
        last = last*(1+np.random.normal(mean, sd, 20)[i])
    return(last)

mc = np.zeros(10000)
for i in range(1,10000):
    mc[i]=get20day(aapl20)
mc.sort()
mc[100]

1 loop, best of 3: 585 ms per loop


Parallel Code

In [211]:
from IPython import parallel 
clients = parallel.Client()
clients.block = True

In [212]:
clients.ids

[0, 1, 2, 3]

In [213]:
import pandas as pd
import numpy as np
import IPython

aapl = pd.read_csv('c:/data/apple.2011.csv', sep=',')
aapl.columns = ['Date', 'Last','Change']
aapl.iloc[0,2] = 0
mean = pd.to_numeric(aapl['Change']).mean()
sd = pd.to_numeric(aapl['Change']).std()



#def get20day(last):
#    for i in range(1,20):
#        last = last*(1+np.random.normal(mean, sd, 20)[i])
#    return(last)

@dview.remote(block=True)
def function():
    mc = np.zeros(2500)
    for j in range(1,2500):
        aapl20 = aapl['Last'].iloc[-1]
        for i in range(1,20):
            aapl20 = aapl20*(1+np.random.normal(mean, sd, 20)[i])
        mc[j] = aapl20
    mc.sort()
    return(mc)

In [214]:
%%timeit
r=function()

1 loop, best of 3: 296 ms per loop


In [215]:
r

[array([   0.        ,  311.04010073,  318.90883257, ...,  521.05828176,
         531.21645473,  531.54781976]),
 array([   0.        ,  313.03370474,  313.84415397, ...,  528.82194792,
         529.02063443,  532.07420262]),
 array([   0.        ,  317.53088179,  317.88348963, ...,  527.73176625,
         527.74177693,  530.20165459]),
 array([   0.        ,  312.45544753,  316.02776222, ...,  515.29023828,
         518.85966957,  519.12020015])]

We got 591 ms when ran as a single process and 301 ms execution time when ran on 4 cores, so there is a clear improvement.